In [ ]:
R.Version()[c('version.string','nickname')] # R 3.2.2 "Fire Safety"

# Fin Loading Analysis

This is an analysis of the aerodynamic loading on the LV3 fins. This is based on the techniques described in [**Tactical Missile Design** by Eugene L. Fleeman (AIAA 2001)][Fleeman2001] ([pdf][Fleeman2001PDF])

Technical drawings of the fins [can be found here][finDrawings].


[Fleeman2001]: https://books.google.com/books?id=iSsfAQAAIAAJ
[Fleeman2001PDF]: https://ia600704.us.archive.org/27/items/TacticalMissileDesignEugeneLFleeman/Tactical%20Missile%20Design%20Eugene%20L%20Fleeman.pdf
[finDrawings]: ../cad/finCan/LongFinFiles/LongFins_ALedge.PDF

## Nomenclature

symbol | variable name | meaning
-------|---------------|--------
$\alpha$ | `alpha` | angle of attack
$\alpha'$ | `alpha_local` | local angle of attack
 | `AR` | aspect ratio
$S_W$ | `area_planform` | exposed area of one fin
$S_Ref$ | `area_wing` | area of two fins, if they were extended to the center line
 | `chord_root` | root chord (if a fin were extended to the center line)
 | `chord_rootExp` | exposed root chord (where the fin meets the module)
 | `chord_tip` | tip chord
 | `OD` | outer diameter of a module
 | `span_semiExp` | exposed semi-span (distance from root to tip)
 | `span_wing` | wing span (distance from tip to tip, if the rocket only had two fins)
 | `sweep_ang00` | zero chord sweep (sweep of the leading edge)
 | `sweep_len00` | zero chord sweep length (distance that the tip lags the root)

In [12]:
# Actual LV3 parameters
parms.LV3 <- expression({
    # Double check these against the technical drawing!
    chord_rootExp <- 18 # inches, exposed root chord
    chord_tip <- 5 # inches, tip chord
    span_semiExp <- 6.42 # inches, exposed semi-span
    sweep_len00 <- 9 # inches, sweep length at 0 percent chord
    sweep_ang00 <- atan(sweep_len00/span_semiExp) # radians, sweep angle at 0 percent chord
    # sweep_ang25 <- 
    area_planform <- span_semiExp*(chord_rootExp+chord_tip)/2
    OD <- 6.6 # inches, "fuselage" outer diameter
    # MAC <- # inches, mean aerodynamic chord

    # The following parameters are not physical measurements on LV3. (no coplanar fins)
    # However, they are used extensively in the engineering equations.
    span_wing <- 2*span_semiExp + OD # inches, wing span. 
    chord_root <- chord_rootExp + (OD/2)*(chord_rootExp-chord_tip)/span_semiExp # inches, root chord
    area_wing <- span_wing*(chord_root+chord_tip)/2 # inches^2, wing area
    AR <- span_wing^2/area_wing # dimensionless
})

# Fleeman's example case:
parms.baseline <- expression({
    
})

In [14]:
eval(parms.LV3)
rm(parms.LV3) # for tidiness
rm(parms.baseline)

In [15]:
sapply( # confirm values of all variables
        ls(), 
        function(x) eval(as.symbol(x))
        )

$AR
[1] 1.309874

$area_planform
[1] 73.83

$area_wing
[1] 288.5114

$chord_root
[1] 24.68224

$chord_rootExp
[1] 18

$chord_tip
[1] 5

$coeff_wingNorm
function (alpha_local) 
{
    ((pi * AR/2 * abs(sin(alpha_local) * cos(alpha_local)) + 
        2 * sin(alpha_local)^2) * (area_planform/area_wing))
}

$OD
[1] 6.6

$span_semiExp
[1] 6.42

$span_wing
[1] 19.44

$sweep_ang00
[1] 0.9511778

$sweep_len00
[1] 9

On pages 30 and 31, the following value is compared to $M^2$:

In [ ]:
1+(8/(pi*AR))^2 # 4.8

So, for a maximum speed of $M=2$, we will always have $M^2<1+(8/(\pi*AR))^2$. Thus, our formula  for fin normal force coefficient is   
$ |(C_N)_{Wing}| = \left((\pi*AR/2)|\sin\alpha'\cos\alpha'|+2\sin^2\alpha'\right)(S_W/S_{Ref})$

In [31]:
AR=2.82
alpha_local=22*2*pi/360

coeff_wingNorm <- function(alpha_local, ...) 
    {(
        (
            pi*AR/2 * abs( sin(alpha_local)*cos(alpha_local) )
            +2*sin(alpha_local)^2
        )
        *(area_planform/area_wing)
    )}
coeff_wingNorm(alpha_local)

[1] 0.4794957